In [68]:
from propar import _PROPAR_MASTERS, instrument
import time
import pandas as pd
pd.options.plotting.backend = 'plotly'

PORT = "COM12"
NODES = [7,8,9,11]
if PORT in _PROPAR_MASTERS:
    master = _PROPAR_MASTERS[PORT]
    try:
        master.stop()
    except Exception as e:
        print(e)
        pass
    del _PROPAR_MASTERS[PORT]     # drop the old master

In [ ]:
# now recreate
mfc = {}
for node in NODES:
    inst = instrument(PORT, address=node)
    if not inst.master.propar.serial.is_open:
        inst.master.start()               # opens COM12 fresh
    inst.writeParameter(10,0)  # ensure that setpoint slope is off
    print(f"{PORT} node {node}. ID {inst.readParameter(1)} current flow {inst.readParameter(205):.3f} sccm")
    mfc[node] = inst

In [20]:
mfc[11].readParameter(4)

0

In [ ]:
inst.master.start()

In [ ]:
def exp_setpoint(elapsed, start: float, finish: float, ppmpm: float):
    "returns expected %, takes percentages of max flow"
    seconds = (finish - start)/ppmpm/60
    sp = start + (finish-start)*elapsed/seconds
    return sp if sp< 100 else 100
def SET(instr, setpoint_p):
    instr.writeParameter(9, int(setpoint_p*320))
def READ(instr):
    return instr.readParameter(8)/320

1746927526.7707493

In [55]:
test_dur = 10  # sec per go
speeds = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
start_p = 10  # starting percent
results = []


SET(mfc[11], start_p)
time.sleep(2)
print(READ(mfc[11]))


for speed in speeds:
    start = time.time()
    while (elapsed := time.time() - start) < test_dur:
        flow_read = READ(mfc[11])
        SET(mfc[11], exp_setpoint(elapsed, start_p, 80, speed))
        results.append({"time_s": elapsed,
                        "speed": speed,
                        "actual": flow_read, 
                        "setp": exp_setpoint(elapsed, start_p, 80, speed)})
    SET(mfc[11], start_p)
    time.sleep(7)
    print(READ(mfc[11]))

10.003125
10.0
10.003125
10.0
10.0
10.03125
10.04375
10.0375


In [67]:
import plotly.graph_objects as go
df = pd.DataFrame(results)
fig = go.Figure()
for speed in speeds:
    df1 = df.loc[df["speed"] == speed]
    #df1["MAPE"] = df1.apply(lambda x: abs(x[2] - x[3])/x[4]*100, raw = True, axis = 1)
    df1 = df1.assign(MAPE = (abs(df1.actual - df1.setp)/df1.setp))
    #fig.add_trace(go.Scatter(x = df1.time_s, y = df1.actual, mode = 'markers', name = speed, legendgroup = speed))
    #fig.add_trace(go.Scatter(x = df1.time_s, y = df1.setp, mode = 'lines', name = speed, legendgroup = speed))
    fig.add_trace(go.Scatter(x = df1.time_s, y = df1.MAPE, name = speed))
    fig.update_layout(plot_bgcolor = 'white',
                      xaxis_gridcolor = 'grey',
                      title = "MFC MAPE upon 1e-3 - 1e3 speed ",
                      yaxis_title = "absolute percent error",
                      xaxis_title = 'elapsed time, seconds',
                      legend_title = "% per second",
                      yaxis_type = 'log',
                      margin=dict(t=60, b=0, l=0, r=0))
fig.show()

In [ ]:
import time
import pandas as pd
pd.options.plotting.backend = 'plotly'

# Parameters
COM_PORT     = "COM12"
NODE         = 11
TEST_DURATION = 10.0  # seconds per baud rate
BAUD_RATES    = [38400, 57600, 115200]

# Connect to MFC
inst = propar.instrument(COM_PORT, address=NODE)
if not inst.master.propar.serial.is_open:
    inst.master.start()

inst.writeParameter(10,0)
inst.writeParameter(9, 3200)
records= []
for baud in BAUD_RATES:
    # Set baud rate
    inst.master.stop()
    inst = propar.instrument(COM_PORT, address = NODE, baudrate= baud)
    if not inst.master.propar.serial.is_open:
        inst.master.start()
    print(f"Testing at baud rate: {baud}")
    
    start = time.time()
    while time.time() - start < TEST_DURATION:
        timestamp = time.time() - start
        
        # Read raw flow (param 9)
        raw_meas = inst.readParameter(8)
        # Write it back as setpoint (param 8)
        inst.writeParameter(9, int(raw_meas*1.01))
        
        records.append({
            "baud_rate": baud,
            "time_s": timestamp,
            "raw_measurement": raw_meas,
            "expected_measurement": int(raw_meas*1.01)
            })
        # minimal delay to query as fast as possible
        time.sleep(0)

In [ ]:
import time
import pandas as pd
from propar import instrument

# Parameters
COM_PORT = "COM12"
NODE = 11
FULL_SCALE_SCCM = 50.0

# Connect to MFC on node 11
inst = instrument(COM_PORT, address=NODE)
if not inst.master.propar.serial.is_open:
    inst.master.start()

def setpoint_slope(oldsp, newsp, seconds):
    slope = int(seconds*32000/(abs(oldsp - newsp)))
    if slope > 30000 or slope <=0:
        raise ValueError("wrong starting params, slope outside 0..30000 bounds")
    else: return slope*10


In [ ]:
setpoint_slope(3200, 3840, 60)

In [ ]:
inst.readParameter(190)

In [ ]:
inst.writeParameter(10,0)
inst.writeParameter(9, 3200)
inst.readParameter(8)

In [ ]:



# 2) Initial setpoint to 10 sccm => raw 10/100*32000 = 3200 citeturn4file2
inst.writeParameter(10,0)
inst.writeParameter(9, 3200)
print("Initial setpoint: 5 sccm (raw 3200)")

# Wait 5 seconds
time.sleep(10)

# 3) Ramp to 20 sccm => raw 20/100*32000 = 6400
# 1) Set setpoint slope to 15 s => writeParameter(10, 15*10) citeturn4file0
inst.writeParameter(10, setpoint_slope(3200, 3840, 60))
inst.writeParameter(9, 3840)
print("Ramping to 10 sccm (raw 6400) over 15 s")

# 4) Collect and print data for 15 seconds
records = []
start = time.time()
while time.time() - start < 15:
    elapsed = time.time() - start
    expected = 5 + (6 - 5) * (elapsed / 60)
    raw_meas = inst.readParameter(8)
    actual_frac = raw_meas / 32000  # scale (0...1) citeturn4file2
    actual_sccm = actual_frac * FULL_SCALE_SCCM
    #print(f"t={elapsed:.1f}s  expected={expected:.2f} sccm  actual={actual_sccm:.2f} sccm")
    records.append({
        "time_s": elapsed,
        "expected_sccm": expected,
        "actual_sccm": actual_sccm
    })
    time.sleep(0)

# 5) Save to DataFrame
df = pd.DataFrame(records).set_index("time_s")
pd.options.plotting.backend = 'plotly'
print(f"dt = {df.index.to_series().diff().dropna().mean()*1000:.0f} ms")
df.plot().update_layout(hovermode = 'x unified')


In [ ]:
# Parameters
COM_PORT = "COM12"
NODE = 11
FULL_SCALE_SCCM = 50.0  # full-scale in sccm for node 11
DURATION = 3600.0         # seconds to record
START_FLOW = 5.0        # sccm
END_FLOW = 6.0          # sccm
RAMP_TIME = 60000.0        # seconds for full ramp (5 → 6 sccm)  # 60000 - 0.001 sccm.min

# Data collection
print(inst.readParameter(8)/32000*50)
inst.writeParameter(10,0)
print(inst.readParameter(10))
inst.writeParameter(9, 3200)
time.sleep(5)
print(inst.readParameter(8)/32000*50)
#inst.writeParameter(10,15000)

records = []
start = time.time()
while (elapsed := time.time() - start) < DURATION:
    # compute manual discrete setpoint along the 1-min ramp
    desired_sccm = START_FLOW + (END_FLOW - START_FLOW) * (elapsed / RAMP_TIME)
    raw_set = int(desired_sccm / 50.0 * 32000)  # parameter 8 expects 0–32000
    inst.writeParameter(9, raw_set)

    # read actual flow (raw 0–32000 on param 9)
    raw_meas = inst.readParameter(8)
    actual_sccm = (raw_meas / 32000.0) * FULL_SCALE_SCCM

    records.append({
        "time_s": elapsed,
        "set_sccm": desired_sccm,
        "actual_sccm": actual_sccm
    })
    # minimal delay to maximize acquisition speed
    time.sleep(0)

# Build DataFrame
df = pd.DataFrame(records).set_index("time_s")
print(f"dt = {df.index.to_series().diff().dropna().mean()*1000:.0f} ms")
df.to_csv("MFC slope testing.csv")

In [ ]:
df.apply(lambda x: abs(x[0] - x[1]), raw=True, axis=1).mean()

In [ ]:
df.plot().update_layout(hovermode = False)

In [ ]:
def MFC_sccm_to_setpoint(sccm: float, node: int) -> int:
    """fsetpoint param #206 in sccm but MFC lets ~56 sccm instead of 50"""
    correction_dict = {7: 0.5652,  # calculated by alicat calibration. coef setpoint% to actual sccm
                       8: 0.5663,
                       9: 0.5652,
                       11: 0.5660,}
    # max_flow_dict = {7: 50,  # sccm
    #                  8: 50,
    #                  9: 50,
    #                  11: 50,}
    FULL_SCALE = 32000
    if sccm > 56: raise ValueError("input sccm is > 56, try lower value")
    # setpoint_p_corrected = sccm/correction_dict[node]
    # setpoint_scale = int(setpoint_p_corrected/100*FULL_SCALE)
    # fsetpoint_sccm = setpoint_p_corrected /100 * max_flow_dict[node]
    return int(sccm/correction_dict[node]/100*FULL_SCALE)

In [ ]:
mfcs[11].setpoint = MFC_sccm_to_setpoint(56, 11)

In [ ]:
mfcs[7].setpoint = MFC_sccm_to_setpoint(0, 7)

In [ ]:
mfcs[7].measure

In [ ]:
mfcs[11].setpoint = 3200

In [ ]:
print(mfcs[11].measure/32000*50)

In [ ]:
mfcs[11].__dict__

In [ ]:
# Cell 2: Set node 9’s setpoint to 50 % of full‐scale
mfc7 = mfcs[7]

# According to the parameter database, 100 % = 32 000 (raw units) :contentReference[oaicite:2]{index=2}:contentReference[oaicite:3]{index=3}
FULL_SCALE = 32000
TARGET_RAW = int(1 * FULL_SCALE)

print(f"Setting node 9 setpoint to {TARGET_RAW} (≈50 %)")
mfc7.setpoint = TARGET_RAW

In [ ]:
# Cell 3: Read back the measured flow and compute percentage
meas_raw = mfc9.measure   # reads parameter 8 (measure), raw 0–32 000 :contentReference[oaicite:6]{index=6}:contentReference[oaicite:7]{index=7}
meas_pct = meas_raw / FULL_SCALE * 100

print(f"Measured on node 9: raw = {meas_raw}, ≈{meas_pct:.1f}%")

In [230]:
# Cell 4: Clean Up & Release COM Port

# 1) Stop the shared master thread and close COM12
for inst in mfcs.values():
    inst.master.stop()

# 2) Remove the master from propar’s cache so future propar.instrument() calls open fresh
import propar
if PORT in propar._PROPAR_MASTERS:
    del propar._PROPAR_MASTERS[PORT]

print("Disconnected all MFCs; COM port released for LabVIEW.")

Disconnected all MFCs; COM port released for LabVIEW.


In [ ]:
# ── Cell 1: Direct set → read loop (no throttling) ────────────────

import time
from datetime import datetime, timezone
import propar

PORT     = "COM12"
NODE     = 9
DURATION = 15.0          # seconds
VALS     = [20.0, 21.0]  # sccm to toggle

# Connect to node 9
mfc9 = propar.instrument(PORT, address=NODE)

# (Optional) ensure RS232‐override control mode so writeParameter(206,…) takes effect:
mfc9.writeParameter(4, 18)

start = time.perf_counter()
count = 0

data = []

while (time.perf_counter() - start) < DURATION:
    sp = VALS[count % 2]
    # Write absolute‐unit setpoint (Fsetpoint, DDE 206)
    mfc9.writeParameter(206, sp)             # :contentReference[oaicite:0]{index=0}:contentReference[oaicite:1]{index=1}
    # Read back the measured flow (Fmeasure, DDE 205)
    meas = mfc9.readParameter(205)           # :contentReference[oaicite:2]{index=2}:contentReference[oaicite:3]{index=3}
    ts   = datetime.now(timezone.utc).isoformat()
    #print(f"{ts}  SP={sp:.1f} sccm  MEAS={meas:.2f} sccm")
    data += [[ts, sp, meas]]
    count += 1

total = time.perf_counter() - start
print(f"\nDirect set-read: {count} cycles in {total:.3f}s → {count/total:.1f} Hz")

# Clean up
#mfc9.master.stop()
#del mfc9

In [ ]:
mfc9.master.stop()

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_ = np.array(data)
set_flow = [float(x) for x in data_[:,1]]
meas_flow = [float(x) for x in data_[:,2]]
index = [pd.to_datetime(x) for x in data_[:,0]]

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x = index, y = set_flow, name='set_flow'))
fig.add_trace(go.Scatter(x = index, y = meas_flow, name='meas_flow'))
fig.show()

In [ ]:
pd.options.plotting.backend = 'plotly'
pd.Series(data = meas_flow, index = index).diff().abs().plot()

In [ ]:
import time
import pandas as pd
from bronkhorst import BronkhorstRS232

# CONFIG
PORT = 'COM12'
NODE = 9
MAX_FLOW_SCCM = 50
FULL_SCALE = 32000
RUN_DURATION = 20  # seconds
SETPOINTS_SCCM = [20, 20.1]

# Connect
mfc = BronkhorstRS232(PORT)
instr = mfc.add_node(NODE)

# Precalculate raw setpoints
setpoints_raw = [int(s / MAX_FLOW_SCCM * FULL_SCALE) for s in SETPOINTS_SCCM]

# Measure loop
log = []
start = time.time()
index = 0

try:
    while time.time() - start < RUN_DURATION:
        # Set alternating setpoint
        instr[1] = setpoints_raw[index % 2]

        # Read raw measured flow (parameter 2)
        raw_meas = instr[2]

        # Log timestamp and raw value
        log.append((time.time(), raw_meas, setpoints_raw[index % 2]))

        index += 1

finally:
    mfc.close()
    print("Disconnected.")

# Convert to DataFrame
df = pd.DataFrame(log, columns=['timestamp', 'raw_meas', 'raw_set'])
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df['meas_sccm'] = df['raw_meas'] / FULL_SCALE * MAX_FLOW_SCCM
df['set_sccm'] = df['raw_set'] / FULL_SCALE * MAX_FLOW_SCCM

# Report
actual_rate = len(df) / RUN_DURATION
print(f"Collected {len(df)} samples at ~{actual_rate:.1f} Hz")
df.to_csv("mfc_raw_log.csv", index=False)


In [ ]:
print(dir(mfc9.master))